<a href="https://colab.research.google.com/github/susmitha-neel/Vnet-medical-segmentation/blob/main/VNet_for_3D_image_segmentaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import glob
import os
import nibabel as nib

import h5py

import tensorflow as tf
import keras
from keras.layers import *
from keras import activations
from keras.models import Model, model_from_json
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import numpy as np

In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"susmithaan","key":"c0d3693623c4f75bd5060ef9d4e535fc"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d andrewmvd/liver-tumor-segmentation

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/liver-tumor-segmentation
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100% 4.83G/4.84G [01:18<00:00, 56.3MB/s]
100% 4.84G/4.84G [01:18<00:00, 66.2MB/s]


In [6]:
!unzip -q {'/content/liver-tumor-segmentation.zip'} -d /content/

In [7]:
liver_tumor_images = '/content/liver-tumor-segmentation.zip'  # Update with the correct folder name

# List all .nii.gz files in the directory
files = glob.glob(os.path.join(liver_tumor_images, '*.nii.gz'))

In [8]:
!mkdir '/content/volumes'
!mkdir '/content/liver_image_dataset/'

In [9]:
import shutil

In [10]:
content_files = os.listdir("/content/")
content_path = "/content/"
for folder in content_files:
  if folder[0] == 'v':
    volume_path = content_path+folder
    volume_files = os.listdir(volume_path)
    for vol_file in volume_files:
      dest_files = os.listdir("/content/volumes")
      if vol_file not in dest_files:
        shutil.move(volume_path+"/"+vol_file,"/content/volumes" )

In [11]:
volume_path = "/content/volumes"
segment_path = "/content/segmentations"

In [12]:
volume_files = os.listdir("/content/volumes")
segment_files = os.listdir("/content/segmentations")
for f in volume_files:
  dest_files = os.listdir("/content/liver_image_dataset")
  if f not in dest_files:
    shutil.move("/content/volumes/"+f,"/content/liver_image_dataset")
for f in segment_files:
  dest_files = os.listdir("/content/liver_image_dataset")
  if f not in dest_files:
    shutil.move("/content/segmentations/"+f,"/content/liver_image_dataset")

In [22]:


import os
import re

import h5py
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import scipy.misc
import scipy.ndimage

INPUT_SIZE = 128  # Input feature width/height
INPUT_DEPTH = 64  # Input depth

""" K. Sahi, S. Jackson, E. Wiebe, G. Armstrong, S. Winters, R. Moore, et al., ”The value of liver windows
settings in the detection of small renal cell carcinomas on unenhanced computed tomography,”
Canadian Association of Radiologists Journal, vol. 65, pp. 71-76, 2014."""
MIN_HU = -160  # Min HU Value
MAX_HU = 240  # Max HU Value


def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(data, key=alphanum_key)


def hu_window(image):
    image[image < MIN_HU] = MIN_HU
    image[image > MAX_HU] = MAX_HU
    image = (image - MIN_HU) / (MAX_HU - MIN_HU)
    image = image.astype("float32")
    return image


def scale_volume(volume, img_depth=INPUT_DEPTH, img_px_size=INPUT_SIZE, hu_value=True):
    if hu_value:
        volume = hu_window(volume)

    size_scale_factor = img_px_size / volume.shape[0]
    depth_scale_factor = img_depth / volume.shape[-1]

    volume = scipy.ndimage.interpolation.rotate(volume, 90, reshape=False)

    vol_zoom = scipy.ndimage.interpolation.zoom(volume, [size_scale_factor, size_scale_factor,
                                                         depth_scale_factor], order=1)

    vol_zoom[vol_zoom < 0] = 0
    vol_zoom[vol_zoom > 1] = 1
    return vol_zoom


def scale_segmentation(segmentation, img_depth=INPUT_DEPTH, img_px_size=INPUT_SIZE):
    size_scale_factor = img_px_size / segmentation.shape[0]
    depth_scale_factor = img_depth / segmentation.shape[-1]

    segmentation = scipy.ndimage.interpolation.rotate(segmentation, 90, reshape=False)

    # Nearest neighbour is used to mantain classes discrete
    seg_zoom = scipy.ndimage.interpolation.zoom(segmentation, [size_scale_factor, size_scale_factor,
                                                               depth_scale_factor], order=0)

    return seg_zoom


def get_data(vol_dir, seg_dir,crop=False):
    volume = nib.load(vol_dir).get_data()
    segmentation = nib.load(seg_dir).get_data()

    if crop:
        aux = []
        for i in range(segmentation.shape[2]):
            if np.sum(segmentation[:, :, i]) > 0:
                aux.append(i)

        volume = volume[:, :, (np.min(aux)-1):(np.max(aux)+1)]
        segmentation = segmentation[:, :, (np.min(aux)-1):(np.max(aux)+1)]

    return volume, segmentation



def create_dataset(path,data, px_size=INPUT_SIZE, slice_count=INPUT_DEPTH, crop=False):
    """Returns dataset with shape (m, z, x, y, n)"""
    dataset = data
    files = sorted_alphanumeric(os.listdir(path))
    segmentations = []
    volumes = []
    for name in files:
        if name[0] == 's':
            segmentations.append(name)
        elif name[0] == 'v':
            volumes.append(name)

    # m = len(volumes)
    if data == 'train':
      m = 0
      n = 2
      r = 2
    elif data == 'val':
      m = 5
      n = 6
      r = 1
    elif data == 'test':
      m = 6
      n = 7
      r = 1
    if crop:
        print("Creating Cropped Data Set:")
    else:
        print("Creating Data Set:")

    slices = []
    print("0/%i (0%%)" % r)
    for i in range(m,n):
        volume, segmentation = get_data(path + volumes[i], path + segmentations[i], crop)

        if volume.shape[0] == 0 or volume.shape[1] == 0 or volume.shape[2] == 0:
          continue
        else:
          volume = scale_volume(volume, slice_count, px_size)
        if segmentation.shape[0] == 0 or segmentation.shape[1] == 0 or segmentation.shape[2] == 0:
          continue
        else:
          segmentation = scale_segmentation(segmentation, slice_count, px_size)
        slices.append([volume, segmentation])
        print("%i/%i (%i%%)" % (i-m+1, r, ((i-m+1)/r*100)))

    dataset = np.array(slices)

    print("Dataset finished with shape:")
    print(dataset.shape)

    return dataset


def write_dataset(data_set, path):
    h5f = h5py.File(path, 'w')
    h5f.create_dataset('data', data=np.expand_dims(data_set[:, 0, :, :, :], -1))
    truth = np.expand_dims(data_set[:, 1, :, :, :], -1)
    truth = divide_segmentation(truth)
    h5f.create_dataset('truth', data=truth)
    h5f.close()
    print("Dataset saved @ %s" % path)


def divide_segmentation(segmentation):
    layer1 = np.copy(segmentation)
    layer2 = np.copy(segmentation)
    background = np.copy(segmentation)

    layer1[layer1==2] = 1
    layer2[layer2==1] = 0
    layer2[layer2 > 0] = 1
    background[background==1] = 5
    background[background == 0] = 1
    background[background>1] = 0
    # np.concatenate((layer1, layer2, background), axis=-1)
    # return layer1, layer2, background
    # a = np.concatenate((layer1, layer2, background), axis=-1)
    # return a
    return layer1  # liver


data_dir = "/content/liver_image_dataset/"
train_dir = data_dir + "train/"
val_dir = data_dir + "val/"
test_dir = data_dir+"/test"
save_dir = "/content/"

print("Obtaining Training Data:")
train_set = create_dataset(data_dir,'train', crop=True)
write_dataset(train_set, save_dir + "train_data.h5")

print("Obtaining Validation Data:")
val_set = create_dataset(data_dir, 'val', crop=True)
write_dataset(val_set, save_dir + "val_data.h5")

print("Obtaining Test Data:")
test_set = create_dataset(data_dir,'test',crop=True)
write_dataset(test_set, save_dir + "test_data.h5")

Obtaining Training Data:
Creating Cropped Data Set:
0/2 (0%)


<ipython-input-22-76c4939084a6>:66: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  volume = nib.load(vol_dir).get_data()
<ipython-input-22-76c4939084a6>:67: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  segmentation = nib.load(seg_dir).get_data()
<ipython-input-22-76c4939084a6>:42: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  volume = scipy.ndimage.interpolation.rotate(volume,

1/2 (50%)


<ipython-input-22-76c4939084a6>:67: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  segmentation = nib.load(seg_dir).get_data()
<ipython-input-22-76c4939084a6>:42: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  volume = scipy.ndimage.interpolation.rotate(volume, 90, reshape=False)
<ipython-input-22-76c4939084a6>:44: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  vol_zoom = scipy.ndimage.interpolation.zoom(volume, [size_scale_factor, size_scale_factor,
<ipython-input-22-76c4939084a6>:56: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` na

2/2 (100%)
Dataset finished with shape:
(2, 2, 128, 128, 64)
Dataset saved @ /content/train_data.h5
Obtaining Validation Data:
Creating Cropped Data Set:
0/1 (0%)


<ipython-input-22-76c4939084a6>:67: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  segmentation = nib.load(seg_dir).get_data()
<ipython-input-22-76c4939084a6>:42: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  volume = scipy.ndimage.interpolation.rotate(volume, 90, reshape=False)
<ipython-input-22-76c4939084a6>:44: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  vol_zoom = scipy.ndimage.interpolation.zoom(volume, [size_scale_factor, size_scale_factor,
<ipython-input-22-76c4939084a6>:56: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` na

1/1 (100%)
Dataset finished with shape:
(1, 2, 128, 128, 64)
Dataset saved @ /content/val_data.h5
Obtaining Test Data:
Creating Cropped Data Set:
0/1 (0%)


<ipython-input-22-76c4939084a6>:67: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  segmentation = nib.load(seg_dir).get_data()
<ipython-input-22-76c4939084a6>:42: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  volume = scipy.ndimage.interpolation.rotate(volume, 90, reshape=False)
<ipython-input-22-76c4939084a6>:44: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  vol_zoom = scipy.ndimage.interpolation.zoom(volume, [size_scale_factor, size_scale_factor,
<ipython-input-22-76c4939084a6>:56: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` na

1/1 (100%)
Dataset finished with shape:
(1, 2, 128, 128, 64)
Dataset saved @ /content/test_data.h5


<ipython-input-22-76c4939084a6>:59: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  seg_zoom = scipy.ndimage.interpolation.zoom(segmentation, [size_scale_factor, size_scale_factor,


In [27]:


f = h5py.File("/content/train_data.h5", 'r')
input_train = f['data'][...]
label_train = f['truth'][...]
f.close()


In [28]:

f = h5py.File("/content/val_data.h5", 'r')
input_val = f['data'][...]
label_val = f['truth'][...]
f.close()


In [29]:

f = h5py.File("/content/val_data.h5", 'r')
input_test = f['data'][...]
label_test = f['truth'][...]
f.close()

In [5]:

#basic vnet model

def vnet_implementation(img_height, img_width, img_depth, n_channels, num_classes):

  #Inputs
  #img_height,img_width = height and width of the image, img_depth = depth(or no. of slices in image), channels = 1 for 3D medical images
  inputs = Input((img_height, img_width, img_depth, n_channels))

  #Downsampling

  #input convolution
  inp_conv = Conv3D(16, (5,5,5), padding = 'same', kernel_initializer='he_normal')(inputs)
  inp_prelu = keras.layers.PReLU()(inp_conv)

  #step_down_1

  conv1 = Conv3D(16, (5,5,5), padding = 'same', kernel_initializer = 'he_normal')(inp_prelu)
  prelu1 = keras.layers.PReLU()(conv1)
  d1 = Conv3D(32, kernel_size = 2, strides = 2, padding = 'same')(prelu1)

  #step_down_2
  conv2 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(d1)
  prelu2 = keras.layers.PReLU()(conv2)
  conv2 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu2)
  prelu2 = keras.layers.PReLU()(conv2)
  d2 = Conv3D(64, kernel_size = 2, strides = 2, padding = 'same')(prelu2)

  #bottleneck
  conv3 = Conv3D(64, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(d2)
  prelu3 = keras.layers.PReLU()(conv3)
  conv3 = Conv3D(64, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu3)
  prelu3 = keras.layers.PReLU()(conv3)

  #step_up_1
  up1 = Conv3DTranspose(32, kernel_size = 2, strides = 2, padding = 'same', kernel_initializer= 'he_normal')(prelu3)
  up1 = concatenate([up1,prelu2])
  conv4 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(up1)
  prelu4 = keras.layers.PReLU()(conv4)
  conv4 = Conv3D(32, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu4)
  prelu4 = keras.layers.PReLU()(conv4)

  #step_up_2
  up2 = Conv3DTranspose(16, kernel_size = 2, strides = 2, padding = 'same', kernel_initializer= 'he_normal')(prelu4)
  up2 = concatenate([up2,prelu1])
  conv5 = Conv3D(16, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(up2)
  prelu5 = keras.layers.PReLU()(conv5)
  conv5 = Conv3D(16, kernel_size = 5, padding = 'same', kernel_initializer = 'he_normal')(prelu5)
#  prelu5 = keras.layers.PReLU()(conv5)

  #output layer
  outputs = Conv3D(1, (1, 1, 1), activation='softmax')(conv5)


  model = Model(inputs=[inputs], outputs=[outputs])
  #compile model outside of this function to make it flexible.
  model.summary()
  return model

In [35]:
!pip install segmentation-models-3D
import segmentation_models_3D as sm
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25]))
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = ['accuracy']

LR = 0.0001
optim = keras.optimizers.Adam(LR)

In [7]:
model = vnet_implementation(128, 128, 64, 1,29)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 64,   │              0 │ -                      │
│                           │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d (Conv3D)           │ (None, 128, 128, 64,   │          2,016 │ input_layer[0][0]      │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu (PReLU)           │ (None, 128, 128, 64,   │     16,777,216 │ conv3d[0][0]           │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_1 (Conv3D)         │ (None, 128, 128, 64,   │         32,016 │ p_re_lu[0][0]          │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_1 (PReLU)         │ (None, 128, 128, 64,   │     16,777,216 │ conv3d_1[0][0]         │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_2 (Conv3D)         │ (None, 64, 64, 32, 32) │          4,128 │ p_re_lu_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_3 (Conv3D)         │ (None, 64, 64, 32, 32) │        128,032 │ conv3d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_2 (PReLU)         │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_4 (Conv3D)         │ (None, 64, 64, 32, 32) │        128,032 │ p_re_lu_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_3 (PReLU)         │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_5 (Conv3D)         │ (None, 32, 32, 16, 64) │         16,448 │ p_re_lu_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_6 (Conv3D)         │ (None, 32, 32, 16, 64) │        512,064 │ conv3d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_4 (PReLU)         │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_7 (Conv3D)         │ (None, 32, 32, 16, 64) │        512,064 │ p_re_lu_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_5 (PReLU)         │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_transpose          │ (None, 64, 64, 32, 32) │         16,416 │ p_re_lu_5[0][0]        │
│ (Conv3DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concaten

 Total params: 71,041,457 (271.00 MB)

 Trainable params: 71,041,457 (271.00 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print(model.input_shape)
print(model.output_shape)

(None, 128, 128, 64, 1)
(None, 128, 128, 64, 1)


In [20]:
model_dir = "/content/vnet_weights.weights.h5"
callbacks = list()
callbacks.append(ModelCheckpoint(model_dir, monitor='val_loss', save_weights_only=True, save_best_only=True))
callbacks.append(CSVLogger(model_dir + "training.log", append=True))
callbacks.append(ReduceLROnPlateau(factor=0.5, patience=2, verbose=1))
callbacks.append(EarlyStopping(patience=2))

In [30]:
x_train, y_train = input_train, label_train
x_val, y_val = input_val, label_val

In [36]:
model.compile(optimizer = Adam(learning_rate=1e-8), loss = 'binary_crossentropy', metrics = metrics)

model.fit(x_train, y_train,
          batch_size=1, epochs=2,
          validation_data=(x_val, y_val), callbacks=callbacks)

Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 128, 128, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 366s 157s/step - accuracy: 0.0790 - loss: 0.6613 - val_accuracy: 0.0404 - val_loss: 0.6549 - learning_rate: 1.0000e-08
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 314s 153s/step - accuracy: 0.0790 - loss: 0.6613 - val_accuracy: 0.0404 - val_loss: 0.6549 - learning_rate: 1.0000e-08


In [42]:
model_json = model.to_json()


In [45]:
keras.models.save_model(model, '/content/model.hdf5')

In [38]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("vnet_weights.weights.h5")

In [41]:



def load_dataset(path, h5=True):
    #print("Loading dataset... Shape:")
    file = h5py.File(path, 'r')
    if h5:
        data = file.get('data')
        truth = file.get('truth')
    else:
        data = file.get('data').value
        truth = file.get('truth').value
    # print(data.shape)
    return data, truth



def predict(test, model):
    predictions = []
    m = test.shape[0]
    print('Starting predictions:')
    print("0/%i (0%%)" % m)

    for i in range(m):
        image = test[i][np.newaxis, :, :, :]
        prediction = model.predict(image, steps=1)
        prediction = np.squeeze(prediction)
        predictions.append(prediction)
        print("%i/%i (%i%%)" % (i + 1, m, ((i + 1) / m * 100)))

    predictions = np.array(predictions)

    print('Predictions obtained with shape:')
    print(predictions.shape)
    return predictions


def write_predictions(predicitons, path):
    h5f = h5py.File(path, 'w')
    h5f.create_dataset(name='predictions', data=predicitons)
    h5f.close()
    print('Predictions saved to ' + path)


save_dir = "/content/"
test_dir = save_dir + "val_data.h5"
weights_dir = save_dir+"vnet_weights." + "weights.h5"

x_test, y_test = load_dataset(test_dir)
model = vnet_implementation(128, 128, 64, 1,1)
model.load_weights(weights_dir)

predictions = predict(x_test, model)

predictions = np.array(np.split(predictions, x_test.shape[0], axis=0))

write_predictions(predictions, save_dir+"predictions_vnet.h5")


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 128, 128, 64,   │              0 │ -                      │
│ (InputLayer)              │ 1)                     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_39 (Conv3D)        │ (None, 128, 128, 64,   │          2,016 │ input_layer_3[0][0]    │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_27 (PReLU)        │ (None, 128, 128, 64,   │     16,777,216 │ conv3d_39[0][0]        │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_40 (Conv3D)        │ (None, 128, 128, 64,   │         32,016 │ p_re_lu_27[0][0]       │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_28 (PReLU)        │ (None, 128, 128, 64,   │     16,777,216 │ conv3d_40[0][0]        │
│                           │ 16)                    │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_41 (Conv3D)        │ (None, 64, 64, 32, 32) │          4,128 │ p_re_lu_28[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_42 (Conv3D)        │ (None, 64, 64, 32, 32) │        128,032 │ conv3d_41[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_29 (PReLU)        │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_42[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_43 (Conv3D)        │ (None, 64, 64, 32, 32) │        128,032 │ p_re_lu_29[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_30 (PReLU)        │ (None, 64, 64, 32, 32) │      4,194,304 │ conv3d_43[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_44 (Conv3D)        │ (None, 32, 32, 16, 64) │         16,448 │ p_re_lu_30[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_45 (Conv3D)        │ (None, 32, 32, 16, 64) │        512,064 │ conv3d_44[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_31 (PReLU)        │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_45[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_46 (Conv3D)        │ (None, 32, 32, 16, 64) │        512,064 │ p_re_lu_31[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ p_re_lu_32 (PReLU)        │ (None, 32, 32, 16, 64) │      1,048,576 │ conv3d_46[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv3d_transpose_6        │ (None, 64, 64, 32, 32) │         16,416 │ p_re_lu_32[0][0]       │
│ (Conv3DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6        

 Total params: 71,041,457 (271.00 MB)

 Trainable params: 71,041,457 (271.00 MB)

 Non-trainable params: 0 (0.00 B)

Starting predictions:
0/1 (0%)


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 128, 128, 64, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
1/1 (100%)
Predictions obtained with shape:
(1, 128, 128, 64)
Predictions saved to /content/predictions_vnet.h5
